In [29]:
import uproot
import awkward as ak
import vector
import os

import matplotlib.pyplot as plt
import numpy as np


In [ ]:
muon_vars = ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge", "Muon_pfRelIso03_all", "Muon_pfRelIso04_all"]
jet_vars = ["FatJet_pt", "FatJet_eta", "FatJet_phi", "FatJet_mass", "FatJet_nConstituents", "FatJet_tau1", "FatJet_tau2"]

In [32]:
path_to_skimmed_nanaod = "/pscratch/sd/r/rmastand/dimuonAD/skimmed_root_files/"

all_muon_vars = {np.empty(1,) for mv in muon_vars}

for filename in os.listdir(path_to_skimmed_nanaod):
    print(filename)
    with open(os.path.join(path_to_skimmed_nanaod, filename), "r") as f:
        
        #open root file
        infile = uproot.open(f)
                                         
        # get events
        events = loc_root_file["Events;1"]
        
        # extract muon vars
        for mv in muon_vars:
            all_muon_vars[mv] = np.concatenate(all_muon_vars[mv], events[mv].array())
            print(all_muon_vars[mv].shape, events[mv].array().shape)
            

nano_data2016_1-1_Skim.root


FileNotFoundError: [Errno 2] No such file or directory: '/global/u1/r/rmastand/dimuonAD/NanoAODTools/nano_data2016_1-1_Skim.root'

In [ ]:
# make filters for true muons, amuons
muon_filter = muon_vars["Muon_charge"] == -1
amuon_filter = muon_vars["Muon_charge"] == 1

num_muon = np.sum(muon_filter, axis = 1)
num_amuon = np.sum(amuon_filter, axis = 1)

# take events with at least 1 mu-amu pair
event_passes = (num_muon > 0) & (num_amuon > 0)


filtered_muon_vars = {}
filtered_amuon_vars = {}

for mv in muon_vars:

    loc_data_mu = all_muon_vars[mv][muon_filter][event_passes]
    loc_data_amu = all_muon_vars[mv][amuon_filter][event_passes]

    # get data corresponding to the hardest (a)muon
    # seems that muons are sorted by decreasing pt, so this is easy
    filtered_muon_vars[mv] = [event[0] for event in loc_data_mu]
    filtered_amuon_vars[mv] = [event[0] for event in loc_data_amu]
    


In [33]:
bins = {"pt":np.linspace(0, 100, 100), 
       "eta":np.linspace(-4, 4, 100),
       "phi":np.linspace(-3.2, 3.2, 100)
       }

nice_labels = {"pt":"$p_T$", 
       "eta":"$\eta$", 
       "phi":"$\phi$", 
       }


for key in keys:

    plt.figure()
    plt.hist(filtered_muons[key], bins = bins[key], histtype = "step", label = "$\mu$", color = "purple")
    plt.hist(filtered_amuons[key], bins = bins[key], histtype = "step", label = "$\overline{\mu}$", color = "green")
    plt.xlabel(nice_labels[key])
    plt.ylabel("Counts")
    plt.legend()
    plt.show()



NameError: name 'keys' is not defined

In [ ]:

import vector

muon_mass = 0.1056583755 # GeV





mu_E = np.sqrt(muon_mass**2 + (filtered_muons["pt"]*np.cosh(filtered_muons["eta"]))**2)
amu_E = np.sqrt(muon_mass**2 + (filtered_amuons["pt"]*np.cosh(filtered_amuons["eta"]))**2)

mu_px = filtered_muons["pt"]*np.cos(filtered_muons["phi"])
amu_px = filtered_amuons["pt"]*np.cos(filtered_amuons["phi"])

mu_py = filtered_muons["pt"]*np.sin(filtered_muons["phi"])
amu_py = filtered_amuons["pt"]*np.sin(filtered_amuons["phi"])

mu_pz = filtered_muons["pt"]*np.sinh(filtered_muons["eta"])
amu_pz = filtered_amuons["pt"]*np.sinh(filtered_amuons["eta"])

dimu_E = mu_E + amu_E
dimu_px = mu_px + amu_px
dimu_py = mu_py + amu_py
dimu_pz = mu_pz + amu_pz

dimu_M = np.sqrt(dimu_E**2 - dimu_px**2 - dimu_py**2 - dimu_pz**2)

    
